In [ ]:
# %% [markdown]
# # Final Quantum Performance & QNN (Bonus)
# This notebook builds a Hybrid Quantum Neural Network using the 
# Native TensorFlow interface and compares it with classical baselines.

# %%
import logging
# Set TensorFlow logger to only show Errors, hiding Warnings and Info
tf.get_logger().setLevel(logging.ERROR)
import sys
import os
import tensorflow as tf
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# Add src to path
sys.path.append(os.path.abspath('../src'))
from data_processing import process_fraud_data

# %%
# 1. Load and Prepare Data
X_train, X_test, y_train, y_test, _ = process_fraud_data('../data/dataset.csv')

# Convert to TensorFlow tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)

# %%
# 2. Define the Quantum Model (Native TF Interface)
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev, interface="tf", diff_method="backprop")
def qnn_circuit(inputs, weights):
    # Encoding classical data (4 features -> 4 qubits)
    qml.AngleEmbedding(inputs, wires=range(4))
    # Trainable Quantum Layers
    qml.StronglyEntanglingLayers(weights, wires=range(4))
    # Measuring the first qubit
    return qml.expval(qml.PauliZ(0))

# Initialize weights as a trainable TensorFlow Variable
# Shape for 3 layers: (layers, qubits, rotations)
weight_shape = (3, 4, 3)
weights = tf.Variable(
    tf.random.uniform(shape=weight_shape, minval=0, maxval=1, dtype=tf.float32), 
    trainable=True
)

# %%
# 3. Hybrid Training Loop
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)

print("Training Hybrid QNN...")
for step in range(20):  # 20 training iterations
    with tf.GradientTape() as tape:
        # Use a subset of data for faster execution in simulation
        batch_idx = np.random.randint(0, len(X_train), 10)
        batch_X = tf.gather(X_train_tf, batch_idx)
        batch_y = tf.gather(y_train_tf, batch_idx)

        # Forward pass: Map circuit over the batch
        # Convert quantum output range [-1, 1] to probability [0, 1]
        logits = [qnn_circuit(x, weights) for x in batch_X]
        probs = (tf.stack(logits) + 1) / 2
        
        # Calculate Binary Cross-Entropy Loss
        loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(batch_y, probs))

    # Backpropagation
    grads = tape.gradient(loss, [weights])
    optimizer.apply_gradients(zip(grads, [weights]))

    if (step + 1) % 5 == 0:
        print(f"Step {step+1} | Loss: {loss.numpy():.4f}")

# %%
# 4. Evaluation and Comparison
test_logits = [qnn_circuit(x, weights) for x in X_test_tf[:50]] # Testing on subset for speed
test_probs = (tf.stack(test_logits).numpy() + 1) / 2
qnn_auc = roc_auc_score(y_test[:50], test_probs)

print("\n" + "="*40)
print(f"{'MODEL TYPE':<25} | {'AUC-ROC SCORE':<10}")
print("-" * 40)
print(f"{'Classical Random Forest':<25} | {0.9250:.4f}") # Replace with real values from Notebook 1
print(f"{'Standard VQC':<25} | {0.7820:.4f}")         # Replace with real values from train_qml.py
print(f"{'Hybrid Bonus QNN':<25} | {qnn_auc:.4f}")
print("="*40)